In [7]:
import os
import gensim
from collections import Counter
import json
import torch


train_path = "./Dataset/yelp-subset.train.csv"
test_path = "./Dataset/yelp-subset.test.csv"
#setup Glove word embeddings
with open('glove.6B/glove.6B.50d.txt', 'r') as f:
    lines = f.readlines()

glove = dict()

for line in lines:
    items = line.split()
    word = items[0]
    vector = list(map(float, items[1:]))
    glove[word] = vector

def build_vocab(path, min_word_count = 5):
	counter = Counter()

	with open(path, 'r') as f:
		lines = f.readlines()
	processedLines = [gensim.utils.simple_preprocess(sentence,min_len=1) for sentence in lines]
	for line in processedLines:
		counter.update(line)
	#initialise a dictionary or look up table
	word2id = {}
	word2id['<pad>'] = 0
	word2id['<unk>'] = 1
	# include only those in dictionary which have occered more than min word count in the entire data.
	words = [word for word, count in counter.items() if count>min_word_count]

	for i, word in enumerate(words):
		word2id[word] = i+2
	return word2id


word2id = build_vocab(train_path)
print("Dictionary Formed and saved. The length of dictionary is-: ", len(word2id))

Dictionary Formed and saved. The length of dictionary is-:  19595


In [12]:
from torch.utils.data import Dataset, DataLoader
class Dataset_seq(Dataset):
	def __init__(self, word2id, train_path):
		self.word2id = word2id
		self.train_path = train_path
		# read the data and label 
		with open(train_path, 'r') as f:
			lines = f.readlines()
		processedLines = [gensim.utils.simple_preprocess(sentence,min_len=1) for sentence in lines]
		self.data = processedLines
		print(self.data[3])

	def __getitem__(self, index):
		# return the seq and label 
		seq = self.preprocess(self.data[index])
		label = self.label[index]
		return seq, label

	def __len__(self):
		return(len(self.data))

q = Dataset_seq(word2id, train_path)


['i', 'love', 'this', 'place', 'the', 'best', 'dim', 'sum', 'in', 'chandler', 'my', 'service', 'was', 'exceptional', 'i', 'even', 'went', 'on', 'lunchtime', 'on', 'a', 'friday', 'with', 'my', 'boyfriend', 'it', 'was', 'priced', 'fairly', 'good', 'we', 'both', 'gorged', 'ourselves', 'on', 'less', 'than', 'not', 'too', 'sure', 'about', 'the', 'pink', 'tablecloths', 'or', 'the', 'giant', 'lobster', 'tank', 'but', 'overall', 'it', 'was', 'delicious']
